Access relevant field of nifti header with and without nibabel.

In [1]:
from __future__ import division, print_function
import numpy as np
import os
from subprocess import check_output
from os.path import join

In [2]:
FSLBINDIR = join(os.environ.get("FSLDIR", '/usr/share/fsl5.0'), 'bin')
FSLINFO   = join(FSLBINDIR, 'fslinfo')

In [3]:
def nifti_info(filename, tag):
    """Extract value of tag from nifti header of file"""
    info = check_output([FSLINFO, filename], universal_newlines=True)
    fields = [line for line in info.split('\n') if line.startswith(tag)][0].split()
    return fields[-1]


def nifti_dims_fsl(filename):
    """Matrix dimensions of image in nifti file"""
    return tuple([int(float(nifti_info(filename, 'dim%d' % i))) for i in range(1, 5)])


def nifti_pixdims_fsl(filename):
    """Pixel dimensions of image in nifti file"""
    return tuple([float(nifti_info(filename, 'pixdim%d' % i)) for i in range(1, 5)])

In [4]:
NIIFILE = '../test/refin/filtered_func_data.nii.gz'
dims_fsl = nifti_dims_fsl(NIIFILE)
pixdims_fsl = nifti_pixdims_fsl(NIIFILE)
print(dims_fsl, pixdims_fsl)

(64, 64, 34, 180) (3.0, 3.0, 3.5, 2.0)


In [5]:
import nibabel as nib

def nifti_dims_nibabel(filename):
    """Matrix dimensions of image in nifti file"""
    return tuple(nib.load(filename).header['dim'][1:5])

def nifti_pixdims_nibabel(filename):
    """Pixel dimensions of image in nifti file"""
    return tuple(nib.load(filename).header['pixdim'][1:5])

In [6]:
dims_nibabel = nifti_dims_nibabel(NIIFILE)
pixdims_nibabel = nifti_pixdims_nibabel(NIIFILE)
print(dims_nibabel, pixdims_nibabel)

(64, 64, 34, 180) (3.0, 3.0, 3.5, 2.0)


In [7]:
assert dims_nibabel == dims_fsl
assert pixdims_nibabel == pixdims_fsl

In [8]:
%timeit nifti_pixdims_fsl(NIIFILE)
%timeit nifti_pixdims_nibabel(NIIFILE)

10 loops, best of 3: 37.8 ms per loop
1000 loops, best of 3: 908 µs per loop


About an order of magnitude improvement using `nibabel` over `fslinfo`. The time is probably not that significant though.